# NLLB use for translation task 

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,DataCollatorForSeq2Seq,EarlyStoppingCallback, TrainingArguments,AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,TrainerCallback, Trainer, DataCollatorForLanguageModeling, logging
import pandas as pd
import json
from sklearn.metrics import cohen_kappa_score
import torch
from tqdm.notebook import tqdm
import os
from huggingface_hub import login
from datasets import load_dataset

In [30]:
dataset_folder = "inputs/dataset_concatenato.csv"
#dataset_folder = "/kaggle/input/dataset-concatenato/dataset_concatenato.csv""
dataset_cleaned_folder = "inputs/dataset.csv"
#dataset_cleaned_folder ="/kaggle/input/dataset-c/dataset_cleaned.csv"
path_NNLB="Models/nllb/nllb-finetuned-antico-moderno"
#path_NNLB="/kaggle/working/nllb-finetuned-antico-moderno"
path_translations="outputs"
#path_translations="/kaggle/working/translations"



In [14]:
dataset = load_dataset("csv", data_files=dataset_folder)["train"]
dataset = dataset.train_test_split(test_size=0.3)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
os.makedirs(path_NNLB, exist_ok=True)

# Import model and setup language

In [20]:
model_name = "facebook/nllb-200-distilled-600M"
src_lang = tgt_lang = "ita_Latn"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.src_lang = "ita_Latn"
forced_bos_token_id = tokenizer.convert_tokens_to_ids("ita_Latn")

# Tokenize text

In [21]:
max_len=256
def preprocess(example):
    model_inputs = tokenizer(
        example["text"],
        max_length=max_len,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        example["translation"],
        max_length=max_len,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = {
    "train": dataset["train"].map(preprocess, remove_columns=dataset["train"].column_names),
    "test": dataset["test"].map(preprocess, remove_columns=dataset["test"].column_names)
}

Map: 100%|██████████| 89/89 [00:00<00:00, 1420.72 examples/s]


# Realize trainer 

In [22]:
class ClearCUDACacheCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir=path_NNLB,
    eval_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-4,
    num_train_epochs=8,
    predict_with_generate=True,
    load_best_model_at_end=True,
    fp16=True, 
    logging_dir=path_NNLB+"/logs",
    save_total_limit=1,
    seed=42,
    report_to="none",
    metric_for_best_model="eval_loss",
    generation_max_length=max_len,
    generation_num_beams=4
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
def compute_metrics(eval_preds):
    return {} 
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[ClearCUDACacheCallback(),EarlyStoppingCallback(early_stopping_patience=1)]
)

C:\Users\colam\AppData\Local\Temp\ipykernel_30768\2908925833.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Training Steps

In [26]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)
model.save_pretrained(path_NNLB)
tokenizer.save_pretrained(path_NNLB)

Epoch,Training Loss,Validation Loss
1,No log,2.866610
2,No log,0.844395
3,No log,0.409088
4,No log,0.419878


c:\Users\colam\Documents\GitHub\MNLP_HW2\.venv\Lib\site-packages\transformers\modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.40908780694007874, 'eval_runtime': 157.1696, 'eval_samples_per_second': 0.566, 'eval_steps_per_second': 0.286, 'epoch': 4.0}


('Models/nllb/nllb-finetuned-antico-moderno\\tokenizer_config.json',
 'Models/nllb/nllb-finetuned-antico-moderno\\special_tokens_map.json',
 'Models/nllb/nllb-finetuned-antico-moderno\\sentencepiece.bpe.model',
 'Models/nllb/nllb-finetuned-antico-moderno\\added_tokens.json',
 'Models/nllb/nllb-finetuned-antico-moderno\\tokenizer.json')

# inference steps 

In [28]:
df = pd.read_csv(dataset_cleaned_folder)
df.head()

,Author,Date,Region,Sentence
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...


In [ ]:
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(path_NNLB)
model = AutoModelForSeq2SeqLM.from_pretrained(path_NNLB,
    device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
model.eval().to(device)


df["generated_translation"] = ""

# 3. Define translation function
def traduci_seq_to_seq(s):
    src_lang = "ita_Latn"
    tgt_lang = "ita_Latn"

    # imposta la lingua sorgente
    tokenizer.src_lang = src_lang
    inputs = tokenizer(s, return_tensors="pt", truncation=True,max_length=384).to(model.device)
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            use_cache=True,
            num_beams=4,
            forced_bos_token_id=forced_bos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 4. Process with progress bar
results = []
i =0
for s in tqdm(df["Sentence"].tolist()):
    results.append(traduci_seq_to_seq(s))
        
df["generated_translation"] = results


os.makedirs(path_translations, exist_ok=True)
# 5. Save result
translation_file=path_translations+"/dataset_with_translation_NNLB.csv"
df.to_csv(translation_file, index=False)

100%|██████████| 97/97 [01:11<00:00,  1.35it/s]


In [33]:
df.head()

,Author,Date,Region,Sentence,generated_translation
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...,"E d'altra parte, Aiaces era un cavaliere franc..."
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c...","e vendichi ogni colpa, come dice la legge, e p..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...,Non è stata innamorata di altra forza di animo...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...,Se questo gli piace a tutti e se il tempo gli ...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...,Sembra che l'ufficio di questa arte sia dire i...
